In [1]:
from music21 import *
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Activation
import glob
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
notes = []

for file in glob.glob("Jazz/*.mid"):
    print("START")
    midi = converter.parse(file)
    print("midi")
    print(midi)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)
    print("parts")
    print(parts)

    if parts: # file has instrument parts
        print("file has parts")
        notes_to_parse = parts.parts[0].recurse()
        print(notes_to_parse)
    else: # file has notes in a flat structure
        print("file doesnt have parts")
        notes_to_parse = midi.flat.notes
    print("element for loop started")
    for element in notes_to_parse:
        #print(element)
        if isinstance(element, note.Note):
            print(element.pitch)
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            print(element.normalOrder)
            notes.append('.'.join(str(n) for n in element.normalOrder))

START
midi
<music21.stream.Score 0x7f37f3d311d0>
parts
<music21.stream.Score 0x7f37f3f03da0>
file has parts
<music21.stream.iterator.RecursiveIterator for Part:None @:0>
element for loop started
D4
[2, 6, 9]
D2
D2
[2, 6, 9]
A1
F#2
G4
A2
G4
F#2
[2, 6, 9]
A1
G1
D4
D2
D2
[2, 6, 9]
A2
F#2
F#4
A2
F#4
D2
[2, 6, 9]
F#2
A4
F#4
[6, 10, 1]
F#2
F#2
[6, 10, 1]
C#2
B-2
B4
C#3
B4
B-2
[6, 10, 1]
C#2
B1
F#4
F#2
F#2
[6, 10, 1]
C#3
B-2
B-4
C#3
B-4
F#2
[6, 10, 1]
B-2
C#5
D4
[2, 6, 9]
D2
D2
[2, 6, 9]
A1
F#2
G4
A2
G4
F#2
[2, 6, 9]
A1
G1
D4
D2
D2
[2, 6, 9]
A2
F#2
F#4
A2
F#4
D2
[2, 6, 9]
F#2
A4
A3
[9, 1, 4]
A1
A1
[9, 1, 4]
E1
C#2
D4
E2
D4
C#2
[9, 1, 4]
E1
D1
A3
A1
A1
[9, 1, 4]
E2
C#2
C#4
E2
C#4
A1
[9, 1, 4]
C#2
E4
[2, 6, 9]
[2, 6, 9]
[2, 6, 9]
[2, 6, 9]
[6, 10, 1]
[6, 10, 1]
[6, 10, 1]
[6, 10, 1]
D4
[2, 6, 9]
F#4
D4
[2, 6, 9]
D4
[2, 6, 9]
F#4
D4
[2, 6, 9]
F#4
[6, 10, 1]
B-4
F#4
[6, 10, 1]
F#4
[6, 10, 1]
B-4
F#4
[6, 10, 1]
D4
[2, 6, 9]
F#4
D4
[2, 6, 9]
D4
[2, 6, 9]
F#4
D4
[2, 6, 9]
F#4
[6, 10, 1]
B-4
F#4
[6, 

In [3]:
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

n_vocab=len(pitchnames)

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

print(note_to_int)

{'0': 0, '0.3': 1, '0.3.7': 2, '0.4.7': 3, '1': 4, '1.7': 5, '10': 6, '10.0.4': 7, '10.1.3': 8, '10.2': 9, '10.2.4': 10, '10.2.5': 11, '11': 12, '2': 13, '2.3.5': 14, '2.5': 15, '2.5.7': 16, '2.5.7.10': 17, '2.5.7.9.10': 18, '2.5.9': 19, '2.6.9': 20, '3.5.7.9': 21, '3.5.9': 22, '3.7': 23, '4': 24, '4.6.10': 25, '4.7.9.0': 26, '5': 27, '5.9.0': 28, '6': 29, '6.10.1': 30, '7': 31, '7.10.0': 32, '7.10.2': 33, '8.10.0.2.4': 34, '8.9': 35, '9': 36, '9.0.3': 37, '9.0.3.5': 38, '9.1.4': 39, '9.10.2.3': 40, '9.10.2.5': 41, '9.2.3': 42, 'A1': 43, 'A2': 44, 'A3': 45, 'A4': 46, 'B-2': 47, 'B-3': 48, 'B-4': 49, 'B1': 50, 'B4': 51, 'C#2': 52, 'C#3': 53, 'C#4': 54, 'C#5': 55, 'C4': 56, 'C5': 57, 'D1': 58, 'D2': 59, 'D4': 60, 'D5': 61, 'E-4': 62, 'E-5': 63, 'E1': 64, 'E2': 65, 'E3': 66, 'E4': 67, 'E5': 68, 'F#2': 69, 'F#4': 70, 'F2': 71, 'F3': 72, 'F4': 73, 'G#4': 74, 'G1': 75, 'G2': 76, 'G3': 77, 'G4': 78}


In [4]:
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
print(n_vocab)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input/float(n_vocab)
#print(network_input)
network_output = to_categorical(network_output)
print(network_output)

79
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [5]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.2))
#model.add(LSTM(512, return_sequences=True))
#model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [56]:
model.fit(network_input, network_output, epochs=200, batch_size=64)

Epoch 1/200
632/632 [==============================] - 9s - loss: 5.0138     
Epoch 2/200
632/632 [==============================] - 8s - loss: 3.7714     
Epoch 3/200
632/632 [==============================] - 8s - loss: 3.5761     
Epoch 4/200
632/632 [==============================] - 9s - loss: 3.3746     
Epoch 5/200
632/632 [==============================] - 10s - loss: 3.2794    
Epoch 6/200
632/632 [==============================] - 11s - loss: 3.1824    
Epoch 7/200
632/632 [==============================] - 10s - loss: 3.0773    
Epoch 8/200
632/632 [==============================] - 9s - loss: 2.9986     
Epoch 9/200
632/632 [==============================] - 11s - loss: 2.8646    
Epoch 10/200
632/632 [==============================] - 10s - loss: 2.8281    
Epoch 11/200
632/632 [==============================] - 10s - loss: 2.7137    
Epoch 12/200
632/632 [==============================] - 10s - loss: 2.6221    
Epoch 13/200
632/632 [==============================] - 10s -

KeyboardInterrupt: 

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_8 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 100, 256)          65792     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
dense_8 (Dense)              (None, 100, 79)           20303     
Total params: 350,287
Trainable params: 350,287
Non-trainable params: 0
_________________________________________________________________


In [75]:
import random
start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []
print(len(pattern))
# generate 500 notes
for note_index in range(500):
    print(note_index)
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output=numpy.append(prediction_output,result)
    print(pattern)
    pattern=numpy.append(pattern,index/n_vocab)
    
    print(len(pattern))
    pattern = pattern[1:]
    print(pattern)

100
0
[[0.13924051]
 [0.21518987]
 [0.34177215]
 [0.13924051]
 [0.07594937]
 [0.4556962 ]
 [0.07594937]
 [0.15189873]
 [0.03797468]
 [0.03797468]
 [0.32911392]
 [0.03797468]
 [0.03797468]
 [0.03797468]
 [0.32911392]
 [0.03797468]
 [0.03797468]
 [0.32911392]
 [0.39240506]
 [0.03797468]
 [0.        ]
 [0.03797468]
 [0.39240506]
 [0.03797468]
 [0.01265823]
 [0.26582278]
 [0.44303797]
 [0.46835443]
 [0.26582278]
 [0.44303797]
 [0.46835443]
 [0.26582278]
 [0.44303797]
 [0.46835443]
 [0.26582278]
 [0.44303797]
 [0.46835443]
 [0.39240506]
 [0.17721519]
 [0.4556962 ]
 [0.11392405]
 [0.13924051]
 [0.34177215]
 [0.13924051]
 [0.50632911]
 [0.46835443]
 [0.35443038]
 [0.48101266]
 [0.13924051]
 [0.34177215]
 [0.29113924]
 [0.27848101]
 [0.13924051]
 [0.43037975]
 [0.98734177]
 [0.88607595]
 [0.98734177]
 [0.88607595]
 [0.84810127]
 [0.98734177]
 [0.88607595]
 [0.98734177]
 [0.88607595]
 [0.98734177]
 [0.88607595]
 [0.84810127]
 [0.93670886]
 [0.64556962]
 [0.93670886]
 [0.92405063]
 [0.84810127]


[0.03797468 0.03797468 0.03797468 0.32911392 0.03797468 0.03797468
 0.32911392 0.39240506 0.03797468 0.         0.03797468 0.39240506
 0.03797468 0.01265823 0.26582278 0.44303797 0.46835443 0.26582278
 0.44303797 0.46835443 0.26582278 0.44303797 0.46835443 0.26582278
 0.44303797 0.46835443 0.39240506 0.17721519 0.4556962  0.11392405
 0.13924051 0.34177215 0.13924051 0.50632911 0.46835443 0.35443038
 0.48101266 0.13924051 0.34177215 0.29113924 0.27848101 0.13924051
 0.43037975 0.98734177 0.88607595 0.98734177 0.88607595 0.84810127
 0.98734177 0.88607595 0.98734177 0.88607595 0.98734177 0.88607595
 0.84810127 0.93670886 0.64556962 0.93670886 0.92405063 0.84810127
 0.58227848 0.98734177 0.58227848 0.98734177 0.84810127 0.58227848
 0.98734177 0.78481013 0.98734177 0.75949367 0.84810127 0.98734177
 0.58227848 0.64556962 0.86075949 0.7721519  0.98734177 0.88607595
 0.98734177 0.88607595 0.84810127 0.98734177 0.88607595 0.98734177
 0.88607595 0.98734177 0.88607595 0.84810127 0.93670886 0.6455

[0.03797468 0.01265823 0.26582278 0.44303797 0.46835443 0.26582278
 0.44303797 0.46835443 0.26582278 0.44303797 0.46835443 0.26582278
 0.44303797 0.46835443 0.39240506 0.17721519 0.4556962  0.11392405
 0.13924051 0.34177215 0.13924051 0.50632911 0.46835443 0.35443038
 0.48101266 0.13924051 0.34177215 0.29113924 0.27848101 0.13924051
 0.43037975 0.98734177 0.88607595 0.98734177 0.88607595 0.84810127
 0.98734177 0.88607595 0.98734177 0.88607595 0.98734177 0.88607595
 0.84810127 0.93670886 0.64556962 0.93670886 0.92405063 0.84810127
 0.58227848 0.98734177 0.58227848 0.98734177 0.84810127 0.58227848
 0.98734177 0.78481013 0.98734177 0.75949367 0.84810127 0.98734177
 0.58227848 0.64556962 0.86075949 0.7721519  0.98734177 0.88607595
 0.98734177 0.88607595 0.84810127 0.98734177 0.88607595 0.98734177
 0.88607595 0.98734177 0.88607595 0.84810127 0.93670886 0.64556962
 0.93670886 0.92405063 0.84810127 0.58227848 0.98734177 0.58227848
 0.98734177 0.84810127 0.58227848 0.98734177 0.78481013 0.9873

[0.46835443 0.39240506 0.17721519 0.4556962  0.11392405 0.13924051
 0.34177215 0.13924051 0.50632911 0.46835443 0.35443038 0.48101266
 0.13924051 0.34177215 0.29113924 0.27848101 0.13924051 0.43037975
 0.98734177 0.88607595 0.98734177 0.88607595 0.84810127 0.98734177
 0.88607595 0.98734177 0.88607595 0.98734177 0.88607595 0.84810127
 0.93670886 0.64556962 0.93670886 0.92405063 0.84810127 0.58227848
 0.98734177 0.58227848 0.98734177 0.84810127 0.58227848 0.98734177
 0.78481013 0.98734177 0.75949367 0.84810127 0.98734177 0.58227848
 0.64556962 0.86075949 0.7721519  0.98734177 0.88607595 0.98734177
 0.88607595 0.84810127 0.98734177 0.88607595 0.98734177 0.88607595
 0.98734177 0.88607595 0.84810127 0.93670886 0.64556962 0.93670886
 0.92405063 0.84810127 0.58227848 0.98734177 0.58227848 0.98734177
 0.84810127 0.58227848 0.98734177 0.78481013 0.98734177 0.75949367
 0.84810127 0.98734177 0.58227848 0.62025316 0.58227848 0.98734177
 0.98734177 0.58227848 0.64556962 0.7721519  0.79746835 0.7215

[0.34177215 0.29113924 0.27848101 0.13924051 0.43037975 0.98734177
 0.88607595 0.98734177 0.88607595 0.84810127 0.98734177 0.88607595
 0.98734177 0.88607595 0.98734177 0.88607595 0.84810127 0.93670886
 0.64556962 0.93670886 0.92405063 0.84810127 0.58227848 0.98734177
 0.58227848 0.98734177 0.84810127 0.58227848 0.98734177 0.78481013
 0.98734177 0.75949367 0.84810127 0.98734177 0.58227848 0.64556962
 0.86075949 0.7721519  0.98734177 0.88607595 0.98734177 0.88607595
 0.84810127 0.98734177 0.88607595 0.98734177 0.88607595 0.98734177
 0.88607595 0.84810127 0.93670886 0.64556962 0.93670886 0.92405063
 0.84810127 0.58227848 0.98734177 0.58227848 0.98734177 0.84810127
 0.58227848 0.98734177 0.78481013 0.98734177 0.75949367 0.84810127
 0.98734177 0.58227848 0.62025316 0.58227848 0.98734177 0.98734177
 0.58227848 0.64556962 0.7721519  0.79746835 0.72151899 0.79746835
 0.72151899 0.62025316 0.98734177 0.62025316 0.70886076 0.75949367
 0.79746835 0.72151899 0.79746835 0.72151899 0.62025316 0.7974

...

[0.55696203 0.40506329 0.98734177 0.06329114 0.07594937 0.34177215
 0.34177215 0.34177215 0.58227848 0.13924051 0.13924051 0.13924051
 0.34177215 0.06329114 0.13924051 0.07594937 0.13924051 0.13924051
 0.34177215 0.34177215 0.13924051 0.13924051 0.07594937 0.4556962
 0.07594937 0.13924051 0.37974684 0.39240506 0.34177215 0.37974684
 0.10126582 0.37974684 0.37974684 0.07594937 0.39240506 0.03797468
 0.03797468 0.03797468 0.13924051 0.21518987 0.03797468 0.13924051
 0.13924051 0.37974684 0.21518987 0.37974684 0.2278481  0.34177215
 0.2278481  0.21518987 0.44303797 0.2278481  0.26582278 0.2278481
 0.13924051 0.07594937 0.65822785 0.13924051 0.13924051 0.13924051
 0.07594937 0.13924051 0.13924051 0.34177215 0.13924051 0.13924051
 0.07594937 0.07594937 0.13924051 0.34177215 0.13924051 0.
 0.02531646 0.27848101 0.46835443 0.34177215 0.88607595 0.37974684
 0.         0.15189873 0.34177215 0.05063291 0.39240506 0.21518987
 0.07594937 0.34177215 0.34177215 0.21518987 0.21518987 0.13924051
 0.13

 0.07594937 0.13924051 0.13924051 0.34177215]
462
[0.34177215 0.06329114 0.13924051 0.07594937 0.13924051 0.13924051
 0.34177215 0.34177215 0.13924051 0.13924051 0.07594937 0.4556962
 0.07594937 0.13924051 0.37974684 0.39240506 0.34177215 0.37974684
 0.10126582 0.37974684 0.37974684 0.07594937 0.39240506 0.03797468
 0.03797468 0.03797468 0.13924051 0.21518987 0.03797468 0.13924051
 0.13924051 0.37974684 0.21518987 0.37974684 0.2278481  0.34177215
 0.2278481  0.21518987 0.44303797 0.2278481  0.26582278 0.2278481
 0.13924051 0.07594937 0.65822785 0.13924051 0.13924051 0.13924051
 0.07594937 0.13924051 0.13924051 0.34177215 0.13924051 0.13924051
 0.07594937 0.07594937 0.13924051 0.34177215 0.13924051 0.
 0.02531646 0.27848101 0.46835443 0.34177215 0.88607595 0.37974684
 0.         0.15189873 0.34177215 0.05063291 0.39240506 0.21518987
 0.07594937 0.34177215 0.34177215 0.21518987 0.21518987 0.13924051
 0.13924051 0.34177215 0.2278481  0.34177215 0.21518987 0.34177215
 0.34177215 0.34177215

[0.4556962  0.07594937 0.13924051 0.37974684 0.39240506 0.34177215
 0.37974684 0.10126582 0.37974684 0.37974684 0.07594937 0.39240506
 0.03797468 0.03797468 0.03797468 0.13924051 0.21518987 0.03797468
 0.13924051 0.13924051 0.37974684 0.21518987 0.37974684 0.2278481
 0.34177215 0.2278481  0.21518987 0.44303797 0.2278481  0.26582278
 0.2278481  0.13924051 0.07594937 0.65822785 0.13924051 0.13924051
 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215 0.13924051
 0.13924051 0.07594937 0.07594937 0.13924051 0.34177215 0.13924051
 0.         0.02531646 0.27848101 0.46835443 0.34177215 0.88607595
 0.37974684 0.         0.15189873 0.34177215 0.05063291 0.39240506
 0.21518987 0.07594937 0.34177215 0.34177215 0.21518987 0.21518987
 0.13924051 0.13924051 0.34177215 0.2278481  0.34177215 0.21518987
 0.34177215 0.34177215 0.34177215 0.4556962  0.13924051 0.07594937
 0.13924051 0.07594937 0.13924051 0.34177215 0.13924051 0.13924051
 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215 0.20253

[0.39240506 0.03797468 0.03797468 0.03797468 0.13924051 0.21518987
 0.03797468 0.13924051 0.13924051 0.37974684 0.21518987 0.37974684
 0.2278481  0.34177215 0.2278481  0.21518987 0.44303797 0.2278481
 0.26582278 0.2278481  0.13924051 0.07594937 0.65822785 0.13924051
 0.13924051 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215
 0.13924051 0.13924051 0.07594937 0.07594937 0.13924051 0.34177215
 0.13924051 0.         0.02531646 0.27848101 0.46835443 0.34177215
 0.88607595 0.37974684 0.         0.15189873 0.34177215 0.05063291
 0.39240506 0.21518987 0.07594937 0.34177215 0.34177215 0.21518987
 0.21518987 0.13924051 0.13924051 0.34177215 0.2278481  0.34177215
 0.21518987 0.34177215 0.34177215 0.34177215 0.4556962  0.13924051
 0.07594937 0.13924051 0.07594937 0.13924051 0.34177215 0.13924051
 0.13924051 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215
 0.20253165 0.21518987 0.36708861 0.34177215 0.13924051 0.34177215
 0.4556962  0.07594937 0.36708861 0.07594937 0.46835443 0.13924

[0.2278481  0.34177215 0.2278481  0.21518987 0.44303797 0.2278481
 0.26582278 0.2278481  0.13924051 0.07594937 0.65822785 0.13924051
 0.13924051 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215
 0.13924051 0.13924051 0.07594937 0.07594937 0.13924051 0.34177215
 0.13924051 0.         0.02531646 0.27848101 0.46835443 0.34177215
 0.88607595 0.37974684 0.         0.15189873 0.34177215 0.05063291
 0.39240506 0.21518987 0.07594937 0.34177215 0.34177215 0.21518987
 0.21518987 0.13924051 0.13924051 0.34177215 0.2278481  0.34177215
 0.21518987 0.34177215 0.34177215 0.34177215 0.4556962  0.13924051
 0.07594937 0.13924051 0.07594937 0.13924051 0.34177215 0.13924051
 0.13924051 0.13924051 0.07594937 0.13924051 0.13924051 0.34177215
 0.20253165 0.21518987 0.36708861 0.34177215 0.13924051 0.34177215
 0.4556962  0.07594937 0.36708861 0.07594937 0.46835443 0.13924051
 0.34177215 0.37974684 0.07594937 0.37974684 0.34177215 0.34177215
 0.39240506 0.39240506 0.07594937 0.39240506 0.13924051 0.34177

In [76]:
prediction_output

array(['B4', 'G#4', 'F4', 'E4', 'A4', 'G4', 'A4', 'G4', 'E4', 'A4', 'G4',
       'E-4', 'G4', 'D4', 'E4', 'G4', 'A4', 'B-4', 'A4', 'G4', 'G4', 'A4',
       'B4', 'D5', 'E-5', 'C5', 'E-5', 'C5', 'B-4', 'G4', 'B-4', 'C4',
       'D4', 'E-5', 'C5', 'E-5', 'C5', 'B-4', 'E-5', 'C5', 'E-5', 'C5',
       'B-4', 'G4', 'B-4', 'C4', 'D4', 'E-5', 'B-4', 'C5', 'D5', 'G4',
       'F#4', 'G4', 'F#4', 'E4', 'G4', 'F#4', 'G4', 'F#4', 'G4', 'F#4',
       'E4', 'G#4', 'B4', 'G#4', 'F4', 'E4', 'A4', 'G4', 'A4', 'G4', 'E4',
       'A4', 'G4', 'E-4', 'G4', 'D4', 'E4', 'G4', 'A4', 'B-4', 'A4', 'E4',
       'G4', 'A4', 'B4', 'C5', 'B4', 'D5', 'B-4', 'B4', 'A4', 'B-4', 'A4',
       'G4', 'D4', 'B4', 'G4', 'B-4', 'G4', 'D4', 'G4', 'F#4', 'F#4',
       'D4', 'A4', 'G4', 'A4', 'G4', 'D4', 'F#4', 'G4', '2.6.9', 'G4',
       'A4', 'G4', 'G4', 'A4', 'G4', '2.6.9', 'G4', 'D4', 'E4', 'D4',
       'F#4', 'G4', 'F#4', 'G4', 'D4', 'G4', 'E4', 'D2', 'G4', 'A4', 'G4',
       'F#4', 'F#4', 'C5', 'F#4', 'G4', 'B-4', 'E4', '

In [77]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model

for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [79]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'